In [1]:
import torch
import torch.nn as nn
import numpy as np
import hashlib
from datetime import datetime

# ============ Simplified Chaotic Key Generator ============
class SimplifiedKeyGenerator:
    """
    Simpler key generation focusing on effectiveness over complexity
    """
    def __init__(self, seed=None):
        if seed is None:
            seed = int(datetime.now().timestamp() * 1000000) % (2**32)
        self.seed = seed
        np.random.seed(seed)

    def logistic_map(self, x, r=3.99):
        """Classic chaotic map"""
        return r * x * (1 - x)

    def generate_key_sequence(self, length, initial=None):
        """Generate a single chaotic sequence"""
        if initial is None:
            initial = np.random.random()

        sequence = []
        x = initial
        for _ in range(length):
            x = self.logistic_map(x)
            sequence.append(x)

        return np.array(sequence, dtype=np.float32)

    def generate_multiple_keys(self, num_keys=4, length=512):
        """Generate multiple independent key sequences"""
        keys = []
        for i in range(num_keys):
            initial = (self.seed + i * 0.123456) / (2**32)
            key_seq = self.generate_key_sequence(length, initial)
            keys.append(key_seq)

        return keys


# ============ String Utilities ============
class StringProcessor:
    """Convert strings to integers and back (printable ASCII only)"""

    def __init__(self, max_length=128):
        self.max_length = max_length

        # Use only printable ASCII: space(32) to ~(126) = 95 characters
        self.char_to_idx = {}
        self.idx_to_char = {}

        # Special tokens
        self.PAD = 0
        self.START = 1
        self.END = 2

        # Printable ASCII characters
        idx = 3
        for ascii_val in range(32, 127):  # space to ~
            char = chr(ascii_val)
            self.char_to_idx[char] = idx
            self.idx_to_char[idx] = char
            idx += 1

        self.vocab_size = idx  # Should be 98 (3 special + 95 ASCII)

    def encode(self, text):
        """Convert string to indices"""
        if len(text) > self.max_length - 2:
            text = text[:self.max_length - 2]

        indices = [self.START]
        for char in text:
            if char in self.char_to_idx:
                indices.append(self.char_to_idx[char])
            else:
                indices.append(self.char_to_idx[' '])  # Unknown → space
        indices.append(self.END)

        # Pad
        while len(indices) < self.max_length:
            indices.append(self.PAD)

        return torch.LongTensor(indices)

    def decode(self, indices):
        """Convert indices back to string"""
        if isinstance(indices, torch.Tensor):
            indices = indices.cpu().numpy()

        chars = []
        for idx in indices:
            idx = int(idx)
            if idx == self.START:
                continue
            elif idx == self.END:
                break
            elif idx == self.PAD:
                continue
            elif idx in self.idx_to_char:
                chars.append(self.idx_to_char[idx])

        return ''.join(chars)

    def batch_encode(self, texts):
        """Encode a batch of strings"""
        return torch.stack([self.encode(text) for text in texts])

    def batch_decode(self, indices_batch):
        """Decode a batch of indices"""
        return [self.decode(indices) for indices in indices_batch]


# ============ Key Manager ============
class KeyManager:
    """Manages key generation and distribution"""

    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.key_gen = SimplifiedKeyGenerator()

    def generate_keys_for_batch(self, batch_size, seq_length, message_hash=None):
        """Generate keys for a batch of messages"""

        # Use message hash to seed if provided
        if message_hash is not None:
            seed = int(hashlib.sha256(message_hash.encode()).hexdigest()[:8], 16)
            self.key_gen = SimplifiedKeyGenerator(seed=seed)

        # Generate 4 key sequences (one per encryption layer)
        raw_keys = self.key_gen.generate_multiple_keys(num_keys=4, length=seq_length)

        # Convert to tensors and expand for batch
        key_tensors = []
        for key in raw_keys:
            key_tensor = torch.FloatTensor(key).to(self.device)
            key_tensor = key_tensor.unsqueeze(0).expand(batch_size, -1)
            key_tensors.append(key_tensor)

        return {
            'raw_keys': raw_keys,
            'key_tensors': key_tensors,
            'num_keys': len(raw_keys)
        }


# ============ Usage Example ============
if __name__ == "__main__":
    print("="*70)
    print("SIMPLIFIED KEY GENERATOR V2")
    print("="*70)

    # Initialize
    processor = StringProcessor(max_length=128)
    key_manager = KeyManager()

    # Test string processing
    test_msg = "Hello, World! Testing 123."
    print(f"\nOriginal: '{test_msg}'")

    encoded = processor.encode(test_msg)
    print(f"Encoded shape: {encoded.shape}")
    print(f"Encoded (first 20): {encoded[:20].tolist()}")

    decoded = processor.decode(encoded)
    print(f"Decoded: '{decoded}'")
    print(f"Match: {decoded == test_msg}")

    # Test key generation
    print(f"\nVocab size: {processor.vocab_size}")

    batch_size = 4
    seq_len = 128
    keys = key_manager.generate_keys_for_batch(batch_size, seq_len)

    print(f"\nGenerated {keys['num_keys']} key sequences")
    print(f"Each key shape: {keys['key_tensors'][0].shape}")
    print(f"Key 0 sample: {keys['key_tensors'][0][0, :10]}")

    print("\n" + "="*70)

SIMPLIFIED KEY GENERATOR V2

Original: 'Hello, World! Testing 123.'
Encoded shape: torch.Size([128])
Encoded (first 20): [1, 43, 72, 79, 79, 82, 15, 3, 58, 82, 85, 79, 71, 4, 3, 55, 72, 86, 87, 76]
Decoded: 'Hello, World! Testing 123.'
Match: True

Vocab size: 98

Generated 4 key sequences
Each key shape: torch.Size([4, 128])
Key 0 sample: tensor([0.9959, 0.0162, 0.0636, 0.2377, 0.7230, 0.7991, 0.6406, 0.9186, 0.2984,
        0.8354])

